# Scrollable dicom study viewer + Segmentation overlap from XNAT
This code will:

1- Reading Dicoms study (including multiple series, containg multiple images)
2- Creating a jupyter output for Scrrollable dicom viewer, for selected dicom series
3- Add segmentation (downloaded from XNAT) on the image series  


E1 Final create at 20240105 
My context: I coded this on my windows10 and very regular GPU (GTX 1660 and Intel UHD graphic) and Corei7-10750 and 16G Ram. I am coding on VS code and using jupyter notebook during development.

Wehere I left off: I solved the extraction of uid and segmentation arrays from segmentaion

## Dicom Scrollable Viewer only,

### Enviroments and Variables

In [ ]:
#libraaries
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import ipywidgets as widgets
from IPython.display import display, clear_output
from pydicom.multival import MultiValue

#variables (you can find them in code section by searching for '#previously_definied' )
look_only_main_direcotry = False
dicom_directory = r'E:\Data\Ready4XNAT\PanCanAID\PanCanAID_forGP_20231231_AnonymizeClean\1000506'

# You should play with these two variables to get best height of image and jupyter output, for your case.
widget_box_height='400px' #This will deterimine your whole widget heigght, which includes drop down, scroll bar, and CT scan image.
CT_dpi_heigh=40 #This will deterimine your CT scan image height


### Code and Viewer

In [ ]:
#v2 first final and success


def get_dicom_tag_value(dicom_file, tag, default=None):
    '''this function will get the dicom tag from the dicom filde for the given tag/code, so I can get dicom series number'''
    tag_value = dicom_file.get(tag, None)
    if tag_value is None:
        return default
    if isinstance(tag_value, MultiValue):
        return list(tag_value)  # Convert MultiValue to list
    return tag_value.value

def load_dicom_series(directory, look_only_main_direcotry=False):
    '''This code will look into your directory (set look_only_main_direcotry to True if you only want to look in 
    main direcotry and not subfolders within your directory). It will get files with .dcm extenstion. It will create 
    a dictionary for each dicom series, and store all dicoms, under the related dicom series.
    
    !!! if your dicom files have no extension (it is possible in some cases), you should add .dcm at the end of them
    or do some changes in the code below'''
    if look_only_main_direcotry==True:
        study_dict = {}
        for filename in os.listdir(directory):
            if filename.endswith('.dcm'):
                filepath = os.path.join(directory, filename)
                ds = pydicom.dcmread(filepath)
                series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                if series_no not in study_dict:
                    study_dict[series_no] = []
                study_dict[series_no].append(ds)
        
        for study in study_dict.values():
            study.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict
    else:
        study_dict = {}
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith('.dcm'):
                    filepath = os.path.join(root, filename)
                    ds = pydicom.dcmread(filepath)
                    series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                    if series_no not in study_dict:
                        study_dict[series_no] = []
                    study_dict[series_no].append(ds)
        
        for series in study_dict.values():
            series.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict       


def resize_image(image, target_shape):
    '''sinc I am stacking images on top of each other, I should resize them'''
    # Resize an image to the target shape using interpolation
    resized_image = resize(image, target_shape, preserve_range=True, anti_aliasing=True, mode='constant')
    return resized_image

def convert_to_hu(dicom_images):
    '''This will convert dicom datasets (haunsfild units) to numpy array for visualization.
    It will use the most common image shape to resize very image into this most_common shape (e.g. 512x512).'''

    # Convert DICOM images to Hounsfield Units (HU)
    image_shapes = [image.pixel_array.shape for image in dicom_images]

    if len(set(image_shapes)) != 1:
        # Find the most common shape
        most_common_shape = max(set(image_shapes), key=image_shapes.count)
        # Resize images to the most common shape
        images = np.stack([resize_image(image.pixel_array, most_common_shape) for image in dicom_images])
    else:
        images = np.stack([image.pixel_array for image in dicom_images])
    
    images = images.astype(np.float32)

    # Set outside-of-scan pixels to 0
    images[images == -2000] = 0

    # Convert to Hounsfield units (HU)
    for i, image in enumerate(images):
        if hasattr(dicom_images[i], 'RescaleIntercept') and hasattr(dicom_images[i], 'RescaleSlope'):
            intercept = dicom_images[i].RescaleIntercept
            slope = dicom_images[i].RescaleSlope
        else:
            intercept = 0
            slope = 1
        if slope != 1:
            image = slope * image.astype(np.float32)
            image = image.astype(np.float32)
        image += np.float32(intercept)
        images[i] = image
    
    return images

dicom_directory = dicom_directory #previously_definied
study_dict = load_dicom_series(dicom_directory, look_only_main_direcotry=look_only_main_direcotry) #previously_definied the look_nly

'''Here, I am creating a output window to update dicom images as you scroll'''
# Output widget to display slices
output = widgets.Output()
# Adjust the output widget layout
output.layout.height = widget_box_height  # Adjust the height of output window as needed #previously_definied

# Set the default DPI (increase this value if the image is still too small)
plt.rcParams['figure.dpi'] = CT_dpi_heigh  # You can adjust this value as needed (this is size of ct scan image) #previously_definied


# Function to display a slice
def display_slice(slice_num, study_images):
    '''this function is for showing dicom image for a given slice number of study image'''    
    with output:
        clear_output(wait=True)
        # Adjust figsize to match the aspect ratio of your DICOM images
        aspect_ratio = study_images[0].shape[0] / study_images[0].shape[1]
        plt.figure(figsize=(12, (12 * aspect_ratio)+3))
        plt.imshow(study_images[slice_num], cmap='gray')
        plt.title(f'Slice {slice_num}')
        plt.axis('off')
        plt.show()


# Create a slider widget for slice navigation
slice_slider = widgets.IntSlider(
    min=0, 
    max=1,  # temporary max value, will update based on study selection
    step=1, 
    value=0,
    description='Slice'
)

# Dropdown to select DICOM study
study_selector = widgets.Dropdown(
    options=list(study_dict.keys()),
    description='Select Study:',
    disabled=False,
)

# Function to update slices based on selected study
def update_slices(change):
    '''this will update dicom images showin in the output as your scroll'''
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        max_slice_index = len(study_images) - 1
        slice_slider.max = max_slice_index
        slice_slider.value = 0  # Reset to the first slice
        display_slice(0, study_images)

study_selector.observe(update_slices, names='value')

# Observe changes in the slice_slider and update the displayed slice
def on_slice_change(change):
    '''THis function will update scroll bar min and max for the selected dicom series'''
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        display_slice(change['new'], study_images)

slice_slider.observe(on_slice_change, names='value')

# Display the widgets and the output area
display(study_selector)
display(slice_slider)
display(output)

## Dicom Scrollable Viewer plus segmentation overlap

In [ ]:
#libraaries
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import ipywidgets as widgets
from IPython.display import display, clear_output
from pydicom.multival import MultiValue

#variables (you can find them in code section by searching for '#previously_definied' )
look_only_main_direcotry = False
dicom_directory = r'E:\XNAT_BACK\FirstThree_CT_1\SCANS'

# You should play with these two variables to get best height of image and jupyter output, for your case.
widget_box_height='400px' #This will deterimine your whole widget heigght, which includes drop down, scroll bar, and CT scan image.
CT_dpi_heigh=40 #This will deterimine your CT scan image height

segmentation_directory = r'E:\XNAT_BACK\FirstThree_CT_1\ASSESSORS'


In [ ]:

def get_dicom_tag_value(dicom_file, tag, default=None):
    '''this function will get the dicom tag from the dicom filde for the given tag/code, so I can get dicom series number'''
    tag_value = dicom_file.get(tag, None)
    if tag_value is None:
        return default
    if isinstance(tag_value, MultiValue):
        return list(tag_value)  # Convert MultiValue to list
    return tag_value.value

def load_dicom_series(directory, look_only_main_direcotry=False):
    '''This code will look into your directory (set look_only_main_direcotry to True if you only want to look in 
    main direcotry and not subfolders within your directory). It will get files with .dcm extenstion. It will create 
    a dictionary for each dicom series, and store all dicoms, under the related dicom series.
    
    !!! if your dicom files have no extension (it is possible in some cases), you should add .dcm at the end of them
    or do some changes in the code below'''
    if look_only_main_direcotry==True:
        study_dict = {}
        for filename in os.listdir(directory):
            if filename.endswith('.dcm'):
                filepath = os.path.join(directory, filename)
                ds = pydicom.dcmread(filepath)
                series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                if series_no not in study_dict:
                    study_dict[series_no] = []
                study_dict[series_no].append(ds)
        
        for study in study_dict.values():
            study.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict
    else:
        study_dict = {}
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith('.dcm'):
                    filepath = os.path.join(root, filename)
                    ds = pydicom.dcmread(filepath)
                    series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                    if series_no not in study_dict:
                        study_dict[series_no] = []
                    study_dict[series_no].append(ds)
        
        for series in study_dict.values():
            series.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict       


def resize_image(image, target_shape):
    '''sinc I am stacking images on top of each other, I should resize them'''
    # Resize an image to the target shape using interpolation
    resized_image = resize(image, target_shape, preserve_range=True, anti_aliasing=True, mode='constant')
    return resized_image

def convert_to_hu(dicom_images):
    '''This will convert dicom datasets (haunsfild units) to numpy array for visualization.
    It will use the most common image shape to resize very image into this most_common shape (e.g. 512x512).'''

    # Convert DICOM images to Hounsfield Units (HU)
    image_shapes = [image.pixel_array.shape for image in dicom_images]

    if len(set(image_shapes)) != 1:
        # Find the most common shape
        most_common_shape = max(set(image_shapes), key=image_shapes.count)
        # Resize images to the most common shape
        images = np.stack([resize_image(image.pixel_array, most_common_shape) for image in dicom_images])
    else:
        images = np.stack([image.pixel_array for image in dicom_images])
    
    images = images.astype(np.float32)

    # Set outside-of-scan pixels to 0
    images[images == -2000] = 0

    # Convert to Hounsfield units (HU)
    for i, image in enumerate(images):
        if hasattr(dicom_images[i], 'RescaleIntercept') and hasattr(dicom_images[i], 'RescaleSlope'):
            intercept = dicom_images[i].RescaleIntercept
            slope = dicom_images[i].RescaleSlope
        else:
            intercept = 0
            slope = 1
        if slope != 1:
            image = slope * image.astype(np.float32)
            image = image.astype(np.float32)
        image += np.float32(intercept)
        images[i] = image
    
    return images

dicom_directory = dicom_directory #previously_definied
study_dict = load_dicom_series(dicom_directory, look_only_main_direcotry=look_only_main_direcotry) #previously_definied the look_nly

'''Here, I am creating a output window to update dicom images as you scroll'''
# Output widget to display slices
output = widgets.Output()
# Adjust the output widget layout
output.layout.height = widget_box_height  # Adjust the height of output window as needed #previously_definied

# Set the default DPI (increase this value if the image is still too small)
plt.rcParams['figure.dpi'] = CT_dpi_heigh  # You can adjust this value as needed (this is size of ct scan image) #previously_definied


# Function to display a slice
def display_slice(slice_num, study_images):
    '''this function is for showing dicom image for a given slice number of study image'''    
    with output:
        clear_output(wait=True)
        # Adjust figsize to match the aspect ratio of your DICOM images
        aspect_ratio = study_images[0].shape[0] / study_images[0].shape[1]
        plt.figure(figsize=(12, (12 * aspect_ratio)+3))
        plt.imshow(study_images[slice_num], cmap='gray')
        plt.title(f'Slice {slice_num}')
        plt.axis('off')
        plt.show()


# Create a slider widget for slice navigation
slice_slider = widgets.IntSlider(
    min=0, 
    max=1,  # temporary max value, will update based on study selection
    step=1, 
    value=0,
    description='Slice'
)

# Dropdown to select DICOM study
study_selector = widgets.Dropdown(
    options=list(study_dict.keys()),
    description='Select Study:',
    disabled=False,
)

# Function to update slices based on selected study
def update_slices(change):
    '''this will update dicom images showin in the output as your scroll'''
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        max_slice_index = len(study_images) - 1
        slice_slider.max = max_slice_index
        slice_slider.value = 0  # Reset to the first slice
        display_slice(0, study_images)

study_selector.observe(update_slices, names='value')

# Observe changes in the slice_slider and update the displayed slice
def on_slice_change(change):
    '''THis function will update scroll bar min and max for the selected dicom series'''
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        display_slice(change['new'], study_images)

slice_slider.observe(on_slice_change, names='value')

# Display the widgets and the output area
display(study_selector)
display(slice_slider)
display(output)

In [ ]:
def extract_series_number_from_filename(filename):
    # Extract the last part of the filename before the extension, and then remove 'S'
    series_number_part = filename.split('_')[-1].split('.')[0]  # Get the part like 'S5'
    series_number = int(series_number_part.lstrip('S'))  # Remove 'S' and convert to integer
    return series_number



def load_segmentations_dic(segmentation_directory):
    segmentation_dict = {}
    for root, dirs, files in os.walk(segmentation_directory):
        for filename in files:
            if filename.endswith('.dcm'):
                series_number = extract_series_number_from_filename(filename)
                filepath = os.path.join(root, filename)
                ds = pydicom.dcmread(filepath)
                series_description_Radiologist = get_dicom_tag_value(ds, (0x0008, 0x103E), default="Unknown")

                # Initialize the nested dictionary structure
                if series_number not in segmentation_dict:
                    segmentation_dict[series_number] = {}
                if series_description_Radiologist not in segmentation_dict[series_number]:
                    segmentation_dict[series_number][series_description_Radiologist] = []

                # Store the segmentation data
                segmentation_dict[series_number][series_description_Radiologist]=ds

    return segmentation_dict



segmentation_directory = segmentation_directory
segmentation_dict = load_segmentations_dic(segmentation_directory)
segmentation_dict


    

In [ ]:
for series in segmentation_dict.keys():
    print(series, ': ',segmentation_dict[series].keys())
    for segmentor in segmentation_dict[series]:
        ds=segmentation_dict[series][segmentor]
        print(ds)
        

In [ ]:
from matplotlib.figure import Figure

def create_overlay(ct_image, segmentation_mask, alpha=0.5):
    # Assuming ct_image and segmentation_mask are NumPy arrays of the same shape
    fig, ax = plt.subplots()
    ax.imshow(ct_image, cmap='gray')  # Display CT image
    ax.imshow(segmentation_mask, cmap='Greens', alpha=alpha)  # Overlay segmentation
    ax.axis('off')
    plt.close(fig)  # Prevents display of the figure
    return fig

def get_ct_image_by_uid(ct_series, sop_instance_uid):
    for ct_image in ct_series:
        if ct_image.SOPInstanceUID == sop_instance_uid:
            return ct_image
    return None

def get_dicom_tag_value(dicom_file, tag, default=None):
    '''this function will get the dicom tag from the dicom filde for the given tag/code'''
    tag_value = dicom_file.get(tag, None)
    if tag_value is None:
        return default
    if isinstance(tag_value, MultiValue):
        return list(tag_value)  # Convert MultiValue to list
    return tag_value.value


def get_segmentaitondicom_RefSOPInstncUID(ds_of_dicomsegmentation):
    '''this function will store UIDs in a list'''
    refsopUIDs=[]
    #Segmentation_meta=get_dicom_tag_value(ds_of_dicomsegmentation, (0x5200, 0x9230))
    segment_sequence = ds_of_dicomsegmentation.get((0x5200, 0x9230), None) #Per-frame Functional Groups Sequence
    if segment_sequence:
        for segment_item in segment_sequence:
            derivation_image_sequence = segment_item.get((0x0008, 0x9124), None)
            if derivation_image_sequence:
                for derivation_item in derivation_image_sequence:
                    source_image_sequence = derivation_item.get((0x0008, 0x2112), None)
                    if source_image_sequence:
                        for source_item in source_image_sequence:
                            referenced_sop_instance_uid = source_item.get((0x0008, 0x1155), None)
                            if referenced_sop_instance_uid:
                                refsopUIDs.append(referenced_sop_instance_uid)  # Return the UID
    return refsopUIDs

def get_pixelarray(ds_of_dicomsegmentation):
    '''this function will store 3D pixel array '''
    array=ds_of_dicomsegmentation.pixel_array
    return array


def map_slices_to_uids(pixel_array, referenced_sop_instance_uids):
    """
    Maps each slice of a 3D pixel array to the corresponding Referenced SOP Instance UID.

    :param pixel_array: A 3D numpy array of image data.
    :param referenced_sop_instance_uids: A list of Referenced SOP Instance UIDs.
    :return: A dictionary mapping UIDs to 2D image slices, or None if lengths do not match.
    """
    if pixel_array.shape[0] != len(referenced_sop_instance_uids):
        print("Error: The number of slices and the number of UIDs do not match.")
        return None

    uid_to_slice_mapping = {}
    for i in range(len(referenced_sop_instance_uids)):
        uid=referenced_sop_instance_uids[i].value
        two_d_img=pixel_array[i, :, :]
        uid_to_slice_mapping[uid] = pixel_array[i, :, :]

    return uid_to_slice_mapping



def create_study_dict_with_segmentations(study_dict, segmentation_dict):
    warnings = []
    study_dict_added_segmentation = {}

    for series_no in segmentation_dict:
        if series_no in study_dict:
            study_dict_added_segmentation[series_no] = {}
            for segmentor in segmentation_dict[series_no]:
                study_dict_added_segmentation[series_no][segmentor] = {}

                ds_of_dicomsegmentation=segmentation_dict[series_no][segmentor]
                print(ds_of_dicomsegmentation)
                referenced_sop_instance_uid = get_segmentaitondicom_RefSOPInstncUID(ds_of_dicomsegmentation)
                pixelarray=get_pixelarray(ds_of_dicomsegmentation)
                uid_pixel_dic=map_slices_to_uids(pixelarray, referenced_sop_instance_uid)

                    for ref_instance in uid_pixel_dic.keys():
                        sop_instance_uid = ref_instance.ReferencedSOPInstanceUID
                        
                        if ct_image is not None:
                            overlay_image = overlay_segmentation(ct_image.pixel_array, seg_data["PixelData"])
                            study_dict_added_segmentation[series_no][segmentor].append(overlay_image)
        else:
            warnings.append(f'The image series {series_no} was valid in segmentations but were missed in original dicom study.')

    print(warnings)
    return study_dict_added_segmentation,ds_of_dicomsegmentation




a,ds_of_dicomsegmentation=create_study_dict_with_segmentations(study_dict,segmentation_dict)
ds_of_dicomsegmentation

In [ ]:
uid_pixel_dic.keys()

In [ ]:
def get_dicom_tag_value(dicom_file, tag, default=None):
    '''this function will get the dicom tag from the dicom filde for the given tag/code'''
    tag_value = dicom_file.get(tag, None)
    if tag_value is None:
        return default
    if isinstance(tag_value, MultiValue):
        return list(tag_value)  # Convert MultiValue to list
    return tag_value.value


def get_segmentaitondicom_RefSOPInstncUID(ds_of_dicomsegmentation):
    '''this function will store UIDs in a list'''
    refsopUIDs=[]
    #Segmentation_meta=get_dicom_tag_value(ds_of_dicomsegmentation, (0x5200, 0x9230))
    segment_sequence = ds_of_dicomsegmentation.get((0x5200, 0x9230), None) #Per-frame Functional Groups Sequence
    if segment_sequence:
        for segment_item in segment_sequence:
            derivation_image_sequence = segment_item.get((0x0008, 0x9124), None)
            if derivation_image_sequence:
                for derivation_item in derivation_image_sequence:
                    source_image_sequence = derivation_item.get((0x0008, 0x2112), None)
                    if source_image_sequence:
                        for source_item in source_image_sequence:
                            referenced_sop_instance_uid = source_item.get((0x0008, 0x1155), None)
                            if referenced_sop_instance_uid:
                                refsopUIDs.append(referenced_sop_instance_uid)  # Return the UID
    return refsopUIDs

def get_pixelarray(ds_of_dicomsegmentation):
    '''this function will store 3D pixel array '''
    array=ds_of_dicomsegmentation.pixel_array
    return array


def map_slices_to_uids(pixel_array, referenced_sop_instance_uids):
    """
    Maps each slice of a 3D pixel array to the corresponding Referenced SOP Instance UID.

    :param pixel_array: A 3D numpy array of image data.
    :param referenced_sop_instance_uids: A list of Referenced SOP Instance UIDs.
    :return: A dictionary mapping UIDs to 2D image slices, or None if lengths do not match.
    """
    if pixel_array.shape[0] != len(referenced_sop_instance_uids):
        print("Error: The number of slices and the number of UIDs do not match.")
        return None

    uid_to_slice_mapping = {}
    for i in range(len(referenced_sop_instance_uids)):
        uid=referenced_sop_instance_uids[i].value
        two_d_img=pixel_array[i, :, :]
        uid_to_slice_mapping[uid] = pixel_array[i, :, :]

    return uid_to_slice_mapping

# Usage
referenced_sop_instance_uid = get_segmentaitondicom_RefSOPInstncUID(ds_of_dicomsegmentation)
pixelarray=get_pixelarray(ds_of_dicomsegmentation)
uid_pixel_dic=map_slices_to_uids(pixelarray, referenced_sop_instance_uid)

uid_pixel_dic

In [ ]:
referenced_sop_instance_uid[1].value

In [ ]:
def extract_referenced_uids(segmentation_dcm):
    # Extract the referenced SOP Instance UIDs
    # This needs to be adjusted based on the actual structure of your segmentation DICOM
    referenced_uids = []
    for item in segmentation_dcm.ReferencedSeriesSequence:
        for ref_item in item.ReferencedInstanceSequence:
            referenced_uids.append(ref_item.ReferencedSOPInstanceUID)
    return referenced_uids

def get_corresponding_ct_slice(ct_series, sop_instance_uid):
    # Find the CT slice that corresponds to the given SOP Instance UID
    for ct_slice in ct_series:
        if ct_slice.SOPInstanceUID == sop_instance_uid:
            return ct_slice
    return None

def display_slice_with_segmentation(ct_slice, segmentation_mask):
    # Display the CT slice with the segmentation overlay
    plt.imshow(ct_slice.pixel_array, cmap='gray')
    plt.imshow(segmentation_mask, cmap='jet', alpha=0.5)  # Overlay with transparency
    plt.axis('off')
    plt.show()

# Archived Code

## Try 1

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt

# Function to load DICOM images from a directory
def load_dicom_series(directory,):
    files = [pydicom.dcmread(os.path.join(directory, f)) for f in os.listdir(directory) if f.endswith('.dcm')]
    files.sort(key=lambda x: int(x.InstanceNumber))
    return files

# Function to display a single DICOM image
def plot_dicom_image(image):
    plt.imshow(image.pixel_array, cmap=plt.cm.bone)
    plt.axis('off')

# Load DICOM series
directory = r'E:\Data\Ready4XNAT\PanCanAID\PanCanAID_forGP_20231231_AnonymizeClean\1001567'
dicom_series = load_dicom_series(directory)



In [ ]:
from flask import Flask, render_template, jsonify, request
import pydicom
import os
import base64
from io import BytesIO
import matplotlib.pyplot as plt

app = Flask(__name__)

# Function to load DICOM series
def load_dicom_series(directory):
    series = {}
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.dcm'):
                filepath = os.path.join(root, file)
                ds = pydicom.dcmread(filepath)
                series_uid = ds.SeriesInstanceUID
                if series_uid not in series:
                    series[series_uid] = []
                series[series_uid].append(ds)
    for uid in series:
        series[uid].sort(key=lambda x: x.InstanceNumber)
    return series

# Convert DICOM image to base64 for web display
def convert_dicom_to_base64(dicom_image):
    fig = plt.figure(figsize=(12, 12))
    plt.imshow(dicom_image.pixel_array, cmap=plt.cm.bone)
    plt.axis('off')
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    base64_data = base64.b64encode(buf.getvalue()).decode('utf-8')
    return base64_data

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/load-series', methods=['POST'])
def load_series():
    directory = request.form['directory']
    series = load_dicom_series(directory)
    series_info = [{'uid': uid, 'num_images': len(images)} for uid, images in series.items()]
    return jsonify(series_info)

@app.route('/get-image', methods=['POST'])
def get_image():
    series_uid = request.form['series_uid']
    image_index = int(request.form['image_index'])
    series = load_dicom_series('path_to_dicom_directory')[series_uid]
    base64_image = convert_dicom_to_base64(series[image_index])
    return jsonify({'image': base64_image})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!-- templates/index.html -->
<!DOCTYPE html>
<html>
<head>
    <title>DICOM Viewer</title>
</head>
<body>
    <h1>DICOM Series Viewer</h1>
    <input type="text" id="directory" placeholder="Enter DICOM directory path">
    <button onclick="loadSeries()">Load Series</button>
    <select id="seriesSelect" onchange="showImage(0)">
        <!-- Series options will be loaded here -->
    </select>
    <img id="dicomImage" src="" width="500" />
    <button onclick="changeImage(-1)">Previous</button>
    <button onclick="changeImage(1)">Next</button>

    <script>
        let currentSeries = null;
        let currentIndex = 0;

        function loadSeries() {
            const directory = document.getElementById('directory').value;
            fetch('/load-series', {
                method: 'POST',
                body: new URLSearchParams({directory})
            })
            .then(response => response.json())
            .then(data => {
                const seriesSelect = document.getElementById('seriesSelect');
                seriesSelect.innerHTML = '';
                data.forEach(series => {
                    const option = document.createElement('option');
                    option.value = series.uid;
                    option.textContent = `Series UID: ${series.uid} (${series.num_images} images)`;
                    seriesSelect.appendChild(option);
                });
            });
        }

        function showImage(index) {
            currentIndex = index;
            const series_uid = document.getElementById('seriesSelect').value;
            fetch('/get-image', {
                method: 'POST',
                body: new URLSearchParams({series_uid, image_index: currentIndex})
            })
            .then(response => response.json())
            .then(data => {
                document.getElementById('dicomImage').src = 'data:image/png;base64,' + data.image;
            });
        }

        function changeImage(direction) {
            currentIndex += direction;
            showImage(currentIndex);
        }
    </script>
</body>
</html>


## Try 2

https://stackoverflow.com/questions/59539183/trying-to-scroll-through-ct-slices
https://matplotlib.org/3.1.0/gallery/event_handling/image_slices_viewer.html

In [ ]:
### v1
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

class IndexTracker:
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices // 2

        self.im = ax.imshow(self.X[:, :, self.ind], cmap='gray')
        self.update()

    def onscroll(self, event):
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        elif event.button == 'down':
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        self.ax.set_ylabel(f'slice {self.ind}')
        self.im.axes.figure.canvas.draw()

def load_dicom_series(directory):
    # Load all DICOM images from a directory into a Python list
    dicom_images = [pydicom.dcmread(os.path.join(directory, f)) for f in os.listdir(directory) if f.endswith('.dcm')]
    dicom_images.sort(key=lambda x: int(x.InstanceNumber))  # Sort by instance number
    return dicom_images


def resize_image(image, target_shape):
    # Resize an image to the target shape using interpolation
    resized_image = resize(image, target_shape, preserve_range=True, anti_aliasing=True, mode='constant')
    return resized_image

def convert_to_hu(dicom_images):
    # Convert DICOM images to Hounsfield Units (HU)
    image_shapes = [image.pixel_array.shape for image in dicom_images]

    if len(set(image_shapes)) != 1:
        # Find the most common shape
        most_common_shape = max(set(image_shapes), key=image_shapes.count)
        # Resize images to the most common shape
        images = np.stack([resize_image(image.pixel_array, most_common_shape) for image in dicom_images])
    else:
        images = np.stack([image.pixel_array for image in dicom_images])
    
    images = images.astype(np.float32)

    # Set outside-of-scan pixels to 0
    images[images == -2000] = 0

    # Convert to Hounsfield units (HU)
    for i, image in enumerate(images):
        if hasattr(dicom_images[i], 'RescaleIntercept') and hasattr(dicom_images[i], 'RescaleSlope'):
            intercept = dicom_images[i].RescaleIntercept
            slope = dicom_images[i].RescaleSlope
        else:
            intercept = 0
            slope = 1
        if slope != 1:
            image = slope * image.astype(np.float32)
            image = image.astype(np.float32)
        image += np.float32(intercept)
        images[i] = image
    
    return images


# Specify the path to your DICOM directory
dicom_directory = r'E:\Data\Ready4XNAT\PanCanAID\PanCanAID_forGP_20231231_AnonymizeClean\1001567'

# Load DICOM series
dicom_series = load_dicom_series(dicom_directory)

# Convert DICOM images to Hounsfield Units (HU)
hu_images = convert_to_hu(dicom_series)

fig, ax = plt.subplots(1, 1)
tracker = IndexTracker(ax, hu_images)
fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()


In [ ]:
#v2 first final and success
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import ipywidgets as widgets
from IPython.display import display, clear_output
from pydicom.multival import MultiValue

def get_dicom_tag_value(dicom_file, tag, default=None):
    '''this function will get the dicom tag from the dicom filde for the given tag/code'''
    tag_value = dicom_file.get(tag, None)
    if tag_value is None:
        return default
    if isinstance(tag_value, MultiValue):
        return list(tag_value)  # Convert MultiValue to list
    return tag_value.value

def load_dicom_series(directory, look_only_main_direcotry=False):
    if look_only_main_direcotry==True:
        study_dict = {}
        for filename in os.listdir(directory):
            if filename.endswith('.dcm'):
                filepath = os.path.join(directory, filename)
                ds = pydicom.dcmread(filepath)
                series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                if series_no not in study_dict:
                    study_dict[series_no] = []
                study_dict[series_no].append(ds)
        
        for study in study_dict.values():
            study.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict
    else:
        study_dict = {}
        for root, dirs, files in os.walk(directory):
            for filename in files:
                if filename.endswith('.dcm'):
                    filepath = os.path.join(root, filename)
                    ds = pydicom.dcmread(filepath)
                    series_no = get_dicom_tag_value(ds, (0x0020, 0x0011))
                    if series_no not in study_dict:
                        study_dict[series_no] = []
                    study_dict[series_no].append(ds)
        
        for series in study_dict.values():
            series.sort(key=lambda x: int(x.InstanceNumber))

        return study_dict       


def resize_image(image, target_shape):
    # Resize an image to the target shape using interpolation
    resized_image = resize(image, target_shape, preserve_range=True, anti_aliasing=True, mode='constant')
    return resized_image

def convert_to_hu(dicom_images):

    # Convert DICOM images to Hounsfield Units (HU)
    image_shapes = [image.pixel_array.shape for image in dicom_images]

    if len(set(image_shapes)) != 1:
        # Find the most common shape
        most_common_shape = max(set(image_shapes), key=image_shapes.count)
        # Resize images to the most common shape
        images = np.stack([resize_image(image.pixel_array, most_common_shape) for image in dicom_images])
    else:
        images = np.stack([image.pixel_array for image in dicom_images])
    
    images = images.astype(np.float32)

    # Set outside-of-scan pixels to 0
    images[images == -2000] = 0

    # Convert to Hounsfield units (HU)
    for i, image in enumerate(images):
        if hasattr(dicom_images[i], 'RescaleIntercept') and hasattr(dicom_images[i], 'RescaleSlope'):
            intercept = dicom_images[i].RescaleIntercept
            slope = dicom_images[i].RescaleSlope
        else:
            intercept = 0
            slope = 1
        if slope != 1:
            image = slope * image.astype(np.float32)
            image = image.astype(np.float32)
        image += np.float32(intercept)
        images[i] = image
    
    return images

dicom_directory = r'E:\Data\Ready4XNAT\PanCanAID\PanCanAID_forGP_20231231_AnonymizeClean\1000506'
study_dict = load_dicom_series(dicom_directory, look_only_main_direcotry=False)

# Output widget to display slices
output = widgets.Output()
# Adjust the output widget layout
output.layout.height = '400px'  # Adjust the height of output window as needed
# Set the default DPI (increase this value if the image is still too small)
plt.rcParams['figure.dpi'] = 40  # You can adjust this value as needed (this is size of ct scan image)

# Function to display a slice
def display_slice(slice_num, study_images):
    with output:
        clear_output(wait=True)
        # Adjust figsize to match the aspect ratio of your DICOM images
        aspect_ratio = study_images[0].shape[0] / study_images[0].shape[1]
        plt.figure(figsize=(12, (12 * aspect_ratio)+3))
        plt.imshow(study_images[slice_num], cmap='gray')
        plt.title(f'Slice {slice_num}')
        plt.axis('off')
        plt.show()


# Create a slider widget for slice navigation
slice_slider = widgets.IntSlider(
    min=0, 
    max=1,  # temporary max value, will update based on study selection
    step=1, 
    value=0,
    description='Slice'
)

# Dropdown to select DICOM study
study_selector = widgets.Dropdown(
    options=list(study_dict.keys()),
    description='Select Study:',
    disabled=False,
)

# Function to update slices based on selected study
def update_slices(change):
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        max_slice_index = len(study_images) - 1
        slice_slider.max = max_slice_index
        slice_slider.value = 0  # Reset to the first slice
        display_slice(0, study_images)

study_selector.observe(update_slices, names='value')

# Observe changes in the slice_slider and update the displayed slice
def on_slice_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        series_no = study_selector.value
        study_images = convert_to_hu(study_dict[series_no])
        display_slice(change['new'], study_images)

slice_slider.observe(on_slice_change, names='value')



# Display the widgets and the output area
display(study_selector)
display(slice_slider)
display(output)

## Try 3
ITK
https://pyscience.wordpress.com/2014/09/08/dicom-in-python-importing-medical-image-data-into-numpy-with-pydicom-and-vtk/comment-page-1/

In [ ]:
import dicom
import os
import numpy
from matplotlib import pyplot, cm


PathDicom = "./MyHead/"
reader = vtk.vtkDICOMImageReader()
reader.SetDirectoryName(PathDicom)
reader.Update()


# Load dimensions using `GetDataExtent`
_extent = reader.GetDataExtent()
ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]

# Load spacing values
ConstPixelSpacing = reader.GetPixelSpacing()

# Get the 'vtkImageData' object from the reader
imageData = reader.GetOutput()
# Get the 'vtkPointData' object from the 'vtkImageData' object
pointData = imageData.GetPointData()
# Ensure that only one array exists within the 'vtkPointData' object
assert (pointData.GetNumberOfArrays()==1)
# Get the `vtkArray` (or whatever derived type) which is needed for the `numpy_support.vtk_to_numpy` function
arrayData = pointData.GetArray(0)

# Convert the `vtkArray` to a NumPy array
ArrayDicom = numpy_support.vtk_to_numpy(arrayData)
# Reshape the NumPy array to 3D using 'ConstPixelDims' as a 'shape'
ArrayDicom = ArrayDicom.reshape(ConstPixelDims, order='F')